In [1]:
import pandas as pd 

In [2]:
def read_one_col(file):
    """
    Read file with one column. Strip each line.
    Returns col_list, line number
    """
    df = pd.read_csv(file, header=None)
    col1 = list(df.iloc[:, 0])
    col1 = [item.strip() for item in col1]
    num = len(col1)
    return col1, num

In [ ]:
"/SGRNJ06/randd/USER/cjj/celedev/atac/MAESTRO/test/20231020_10x/downsample_R230621016/02.atac/Result/Analysis/downsample_R230621016"

In [ ]:
df_barcode = pd.read_csv("/SGRNJ06/randd/USER/cjj/celedev/atac/MAESTRO/test/20231020_10x/downsample_R230621016/02.atac/Result/QC/downsample_R230621016/singlecell.txt", header=None, sep='\t', names=["barcode", "fragments", "fragments_overlapping_promoter"])

In [ ]:
df_barcode

In [ ]:
cell_barcode = read_one_col(f"/SGRNJ06/randd/USER/cjj/celedev/atac/MAESTRO/test/20231020_10x/downsample_R230621016/02.atac/Result/QC/downsample_R230621016/downsample_R230621016_scATAC_validcells.txt")[0]

In [ ]:
df_cell_barcode = df_barcode[df_barcode["barcode"].isin(cell_barcode)]

In [ ]:
df_cell_barcode

In [ ]:
peak_count_df = f"/SGRNJ06/randd/USER/cjj/celedev/atac/MAESTRO/test/20231020_10x/downsample_R230621016/02.atac/Result/Analysis/downsample_R230621016/downsample_R230621016_peak_count.txt"

In [ ]:
peak_count_df = pd.read_csv(peak_count_df)

In [ ]:
peak_count_df

In [ ]:
pd.merge(df_cell_barcode, peak_count_df).sort_values('peak_count')

In [ ]:
df_barcode

In [ ]:
peak = pd.read_csv("/SGRNJ06/randd/USER/cjj/celedev/atac/MAESTRO/test/20231020_10x/downsample_R230621016/02.atac/Result/Analysis/downsample_R230621016/downsample_R230621016_final_peaks.bed", header=None, sep='\t', names=["chr", "start", "end"])

In [ ]:
frag = pd.read_csv("/SGRNJ06/randd/USER/cjj/celedev/atac/MAESTRO/test/20231020_10x/downsample_R230621016/02.atac/Result/Mapping/downsample_R230621016/fragments_corrected_count_sortedbybarcode.tsv", header=None, sep='\t', names=["chr", "start", "end", "barcode", "count"])

In [ ]:
peak['start'] = peak['start'].astype(str)
peak['end'] = peak['end'].astype(str)

frag['start'] = frag['start'].astype(str)
frag['end'] = frag['end'].astype(str)

In [ ]:
frag = frag.sort_values(['chr','start','end'])

In [ ]:
peak = peak.sort_values(['chr','start','end'])

In [ ]:
frag

In [ ]:
peak

In [ ]:
peak1 = peak[peak['chr'] == 'chr1']

In [ ]:
final_df = pd.DataFrame()

In [ ]:
for index_peak, data_peak in peak.iterrows():
    peak_info = dict(data_peak)
    # print(peak_info)
    frag_chr = frag[frag['chr'] == peak_info['chr']]
    # print(frag_chr)
    frag_overlap_peak = frag_chr[ (frag_chr['start'] >= peak_info['start']) & (frag_chr['end'] <= peak_info['end'])] 
    # print(frag_overlap_peak)
    final_df = pd.concat([final_df, frag_overlap_peak])
    

In [ ]:
final_df_count = final_df.groupby('barcode', as_index=False).agg({'count': 'sum'}).sort_values('count')

In [ ]:
final_df_count[final_df_count['barcode'] == 'ACACATCGATGCGGCT']

In [ ]:
final_df_count

In [ ]:
df_barcode = pd.merge(df_barcode, final_df_count, on='barcode', how='outer').fillna(0)

In [ ]:
df_barcode = df_barcode.rename(columns={'count': 'overlap_peaks'})

In [ ]:
df_barcode['frac_peak'] = round(df_barcode['overlap_peaks'] / df_barcode['fragments'], 4)

In [ ]:
df_barcode['cell_called'] = df_barcode['barcode'].apply(lambda x: True if x in cell_barcode else False)

In [ ]:
df_barcode.to_csv("/SGRNJ06/randd/USER/cjj/celedev/atac/MAESTRO/test/20231020_10x/downsample_R230621016/02.atac/Result/Analysis/downsample_R230621016/cell_qc_metrics.tsv", sep='\t', index=False)

In [ ]:
test = pd.read_csv("/SGRNJ06/randd/USER/cjj/celedev/atac/MAESTRO/test/20231020_10x/downsample_R230621016/02.atac/Result/Analysis/downsample_R230621016/cell_qc_metrics.tsv", sep='\t')

In [ ]:
test

In [ ]:
frac_peak = test[test['cell_called']==True]['frac_peak'].mean()

In [ ]:
frac_peak

In [ ]:
f'{round(frac_peak * 100, 2)}%'

In [ ]:
test[test['cell_called']==True].sort_values('overlap_peaks')

In [ ]:
df = pd.read_csv("/SGRNJ06/randd/USER/cjj/celedev/atac/MAESTRO/test/20231020_10x/downsample_R230621016/02.atac/Result/Analysis/downsample_R230621016/cell_qc_metrics.tsv", sep='\t')

In [ ]:
peak_res = pd.read_csv("/SGRNJ06/randd/USER/cjj/celedev/atac/MAESTRO/test/20231020_10x/downsample_R230621016/02.atac/Result/Analysis/downsample_R230621016/downsample_R230621016_final_peaks.bed", sep='\t', header=None)

In [ ]:
peak_res

In [ ]:
peak_res.shape[0]

In [ ]:
cell_qc_metrics

In [ ]:
meta = pd.read_csv("/SGRNJ06/randd/USER/cjj/celedev/atac/MAESTRO/test/20231020_10x/downsample_R230621016/03.analysis/meta.csv")

In [ ]:
meta

In [ ]:
meta = meta.rename(columns={"Row.names": "barcode"})

In [ ]:
meta

In [ ]:
df

In [ ]:
pd.merge(meta, df)

In [16]:
import pandas as pd 
import sys
import numpy as np 

In [2]:
df_peaks=pd.read_csv("/SGRNJ06/randd/USER/cjj/celedev/atac/MAESTRO/test/20231031sgr/A1017_3_ML_YSPB_WX_R_T7_EDTA5030/02.atac/Result/Analysis/A1017_3_ML_YSPB_WX_R_T7_EDTA5030/A1017_3_ML_YSPB_WX_R_T7_EDTA5030_final_peaks.bed",header=None, sep='\t', names=["chr", "start", "end"])

In [3]:
df_fragments = pd.read_csv("/SGRNJ06/randd/USER/cjj/celedev/atac/MAESTRO/test/20231031sgr/A1017_3_ML_YSPB_WX_R_T7_EDTA5030/02.atac/Result/Mapping/A1017_3_ML_YSPB_WX_R_T7_EDTA5030/fragments_corrected_count_sortedbybarcode.tsv",header=None, sep='\t', names=["chr", "start", "end", "barcode", "count"])

In [4]:
df_fragments

,chr,start,end,barcode,count
0,chr2,93428094,93428159,AACAATAACGTCTCAACAAT,2
1,chr3,68786956,68787033,AACAATAACGTCTCAACAAT,1
2,chr10,86043406,86043558,AACAATAACGTCTCAACGCG,1
3,chr11,79710911,79710960,AACAATAACGTCTCACACAC,1
4,chr10,124814740,124815078,AACAATAACGTCTCACAGTG,1
...,...,...,...,...,...
19469891,chr16,62368656,62368704,TTTTTGTTGTCCGATTTGAC,1
19469892,chr19,21860387,21860415,TTTTTGTTGTCCGATTTGAC,2
19469893,chr3,105895222,105895250,TTTTTGTTGTCCGATTTGAC,1
19469894,chr3,62632471,62632522,TTTTTGTTGTCCGATTTGAC,1


In [13]:
print(str(sys.getsizeof(df_fragments)/1024/1024)+' MB')

3016.240354537964 MB


In [14]:
df = df_fragments

In [ ]:
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    # 计算当前占用的内存 
    start_mem = df.memory_usage(deep=True).sum() / 1024**2
    # 循环每一列
    for col in df.columns:
        # 获取每一列的数据类型
        col_type = df[col].dtypes
        # 如果数据类型属于上面定义的类型之
        if col_type in numerics:
            # 计算该列数据的最小值和最大值 用于我们指定相应的数据类型 
            c_min = df[col].min()
            c_max = df[col].max()

            # 如果 该列的数据类型属于 int 类型，然后进行判断
            if str(col_type)[:3] == 'int':
                # 如果 该列最小的值 大于int8类型的最小值，并且最大值小于int8类型的最大值，则采用int8类型 
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)

                # 同上
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)

                # 同上
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)

                # 同上
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)

            # 否则 则采用 float 的处理方法       
            else:

                if c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)

                else:
                    df[col] = df[col].astype(np.float64)

In [18]:
print(str(sys.getsizeof(df)/1024/1024)+' MB')

2737.721258163452 MB


In [ ]:
final_df = pd.DataFrame()
for _, data_peak in df_peaks.iterrows():
    peak_info = dict(data_peak)
    print(peak_info)
    frag_chr = df_fragments[df_fragments["chr"] == peak_info["chr"]]
    frag_overlap_peak = frag_chr[ (frag_chr["start"] >= peak_info["start"]) & (frag_chr["end"] <= peak_info["end"])] 
    final_df = pd.concat([final_df, frag_overlap_peak])
    final_df_count = final_df.groupby('barcode', as_index=False).agg({'count': 'sum'})
    break

In [ ]:
final_df

In [32]:
from multiprocessing import Pool
import pandas as pd 

In [33]:
df_peaks=pd.read_csv("/SGRNJ06/randd/USER/cjj/celedev/atac/MAESTRO/test/20231031sgr/A1017_3_ML_YSPB_WX_R_T7_EDTA5030/02.atac/Result/Analysis/A1017_3_ML_YSPB_WX_R_T7_EDTA5030/A1017_3_ML_YSPB_WX_R_T7_EDTA5030_final_peaks.bed",header=None, sep='\t', names=["chr", "start", "end"])
df_fragments = pd.read_csv("/SGRNJ06/randd/USER/cjj/celedev/atac/MAESTRO/test/20231031sgr/A1017_3_ML_YSPB_WX_R_T7_EDTA5030/02.atac/Result/Mapping/A1017_3_ML_YSPB_WX_R_T7_EDTA5030/fragments_corrected_count_sortedbybarcode.tsv",header=None, sep='\t', names=["chr", "start", "end", "barcode", "count"])

In [34]:
df_peaks = df_peaks.sort_values(['chr','start','end'])
df_fragments = df_fragments.sort_values(['chr','start','end'])

In [6]:
df_peaks

,chr,start,end
0,chr1,4768517,4768778
1,chr1,4785335,4786194
2,chr1,4807373,4808300
3,chr1,4857419,4858586
4,chr1,5082842,5083595
...,...,...,...
51862,chrY,90761198,90761746
51863,chrY,90798987,90799576
51864,chrY,90800522,90800811
51865,chrY,90805049,90805601


In [7]:
df_peak1 = df_peaks.iloc[:100,:]
df_peak2 = df_peaks.iloc[100:200,:]
df_peak3 = df_peaks.iloc[200:300,:]
df_peak4 = df_peaks.iloc[300:400,:]
df_peak5 = df_peaks.iloc[400:500,:]

In [8]:
df_peak5

,chr,start,end
400,chr1,39530310,39530538
401,chr1,39535408,39536281
402,chr1,39545755,39546359
403,chr1,39555672,39556320
404,chr1,39568722,39569170
...,...,...,...
495,chr1,43500133,43500382
496,chr1,43501233,43501475
497,chr1,43520319,43520843
498,chr1,43521051,43521363


In [35]:
threads = 8
length = df_peaks.shape[0]

In [36]:
chunk_size = length // threads + 1

In [37]:
#chunk_size
chunk_size = 20

In [38]:
df_peak_list = [df_peaks.iloc[ chunk_size*i: chunk_size*(i+1) ,:] for i in range(threads)]

In [ ]:
df_peak_list

In [39]:
# multiprocessing
def get_df(df_peak):
    final_df = pd.DataFrame()
    for _, data_peak in df_peak.iterrows():
        peak_info = dict(data_peak)
        frag_chr = df_fragments[df_fragments["chr"] == peak_info["chr"]]
        frag_overlap_peak = frag_chr[ (frag_chr["start"] >= peak_info["start"]) & (frag_chr["end"] <= peak_info["end"])] 
        final_df = pd.concat([final_df, frag_overlap_peak])
    return final_df

In [ ]:
with Pool(threads) as p:
    results = p.map(get_df, df_peak_list)

In [40]:
with Pool(threads) as p:
    results = p.starmap(get_df, zip(df_peak_list))

In [ ]:
results

In [41]:
len(results)

8

In [42]:
final_df = pd.DataFrame()
for result in results:
    final_df = pd.concat([final_df, result], ignore_index=False)

In [26]:
final_df

,chr,start,end,barcode,count
0,chr1,4768517,4768564,GTAGTTCGTGTCAAATATCG,1
1,chr1,4768517,4768576,GCCAACGATCTTCCGGGGGT,2
2,chr1,4768519,4768566,TTGCGAGCATCAAGCCTATC,1
3,chr1,4768548,4768778,GCTTAGTATCCGGAATTACA,1
4,chr1,4768550,4768613,CAAGGATCGCTATCACGTCC,3
...,...,...,...,...,...
28103,chr1,21963758,21963790,CAGACTTGGATTGGACCCTT,1
28104,chr1,21963758,21963803,GATCTTTCCAATCGCAATTT,2
28105,chr1,21963768,21963808,CAGTACATACGTCGCGGCTT,1
28106,chr1,21963777,21963814,ATTACAAGACGATGTAACTA,1


In [ ]:
final_df.groupby('barcode', as_index=False).agg({'count': 'sum'})

In [ ]:
a = [df_fragments]*8

In [ ]:
a

In [28]:
df_peaks=pd.read_csv("/SGRNJ06/randd/USER/cjj/celedev/atac/MAESTRO/test/20231031sgr/A1017_3_ML_YSPB_WX_R_T7_EDTA5030/02.atac/Result/Analysis/A1017_3_ML_YSPB_WX_R_T7_EDTA5030/A1017_3_ML_YSPB_WX_R_T7_EDTA5030_final_peaks.bed",header=None, sep='\t', names=["chr", "start", "end"])
df_fragments = pd.read_csv("/SGRNJ06/randd/USER/cjj/celedev/atac/MAESTRO/test/20231031sgr/A1017_3_ML_YSPB_WX_R_T7_EDTA5030/02.atac/Result/Mapping/A1017_3_ML_YSPB_WX_R_T7_EDTA5030/fragments_corrected_count_sortedbybarcode.tsv",header=None, sep='\t', names=["chr", "start", "end", "barcode", "count"])

In [91]:
df_barcode = pd.read_csv(f"/SGRNJ06/randd/USER/cjj/celedev/atac/MAESTRO/test/20231031sgr/A1017_3_ML_YSPB_WX_R_T7_EDTA5030/02.atac/Result/QC/A1017_3_ML_YSPB_WX_R_T7_EDTA5030/singlecell.txt",
                                      header=None, sep='\t', names=["barcode", "fragments", "fragments_overlapping_promoter"])

In [ ]:
df_barcode

In [43]:
df_fragments.sort_values(['chr','start','end'], inplace=True)
df_peaks.sort_values(['chr','start','end'], inplace=True)

In [ ]:
df_fragments

In [ ]:
df_peaks

In [ ]:
df_peaks = df_peaks.iloc[ :20, :]

In [ ]:
df_peaks

In [ ]:
final_df = pd.DataFrame()

In [ ]:
        for _, data_peak in df_peaks.iterrows():
            peak_info = dict(data_peak)
            frag_chr = df_fragments[df_fragments["chr"] == peak_info["chr"]]
            frag_overlap_peak = frag_chr[ (frag_chr["start"] >= peak_info["start"]) & (frag_chr["end"] <= peak_info["end"])] 
            final_df = pd.concat([final_df, frag_overlap_peak])

In [ ]:
final_df

In [ ]:
final_df_count = final_df.groupby('barcode', as_index=False).agg({'count': 'sum'})

In [ ]:
final_df_count

In [ ]:
df_barcode = pd.merge(df_barcode, final_df_count, on='barcode', how='outer').fillna(0)
df_barcode = df_barcode.rename(columns={'count': 'overlap_peaks'})
df_barcode['frac_peak'] = round(df_barcode['overlap_peaks'] / df_barcode['fragments'], 4)

In [ ]:
df_barcode[df_barcode["barcode"]=="AACAATACTGTGGTAAGTGT"]

In [ ]:
# index

In [66]:
df_fragments

,chr,start,end,barcode,count
165035,chr1,3003786,3003903,AACAATTGGCTTCTTTTGAC,1
1633690,chr1,3003975,3004005,ACCCTTCAGGATCATGACCC,1
10348822,chr1,3004652,3004699,GAAAAGTGTGCGTACCCGCC,1
2386893,chr1,3004791,3004834,ACTGGTTGCATCCTAGCCCA,2
15793733,chr1,3005277,3005613,TCAACTTAGTGAGCACGTCC,3
...,...,...,...,...,...
9833889,chrY,90828968,90829015,CTTCAGGCGATGTTTCATTC,1
3749131,chrY,90828971,90829014,AGTCTGTTCTAGGCCAGTAC,3
502508,chrY,90828984,90829202,AAGTGTACGTTGCAAGGGTC,2
15917205,chrY,90829080,90829202,TCATTCATGGTGGAACACAC,1


In [67]:
df_peaks

,chr,start,end
0,chr1,4768517,4768778
1,chr1,4785335,4786194
2,chr1,4807373,4808300
3,chr1,4857419,4858586
4,chr1,5082842,5083595
...,...,...,...
51862,chrY,90761198,90761746
51863,chrY,90798987,90799576
51864,chrY,90800522,90800811
51865,chrY,90805049,90805601


In [68]:
threads = 8
length = df_peaks.shape[0]

In [69]:
chunk_size = length // threads + 1

In [70]:
chunk_size
#chunk_size = 20

6484

In [71]:
df_peak_list = [df_peaks.iloc[ chunk_size*i: chunk_size*(i+1) ,:] for i in range(threads)]

In [72]:
# multiprocessing
def get_df(df_peak):
    index_res = set()
    for _, data_peak in df_peak.iterrows():
        peak_info = dict(data_peak)
        frag_chr = df_fragments[df_fragments["chr"] == peak_info["chr"]]
        frag_overlap_peak = frag_chr[ (frag_chr["start"] >= peak_info["start"]) & (frag_chr["end"] <= peak_info["end"])] 
        index_res.update(set(frag_overlap_peak.index))
    return index_res

In [73]:
with Pool(threads) as p:
    results = p.starmap(get_df, zip(df_peak_list))

In [74]:
results[0]

1239490

In [75]:
import itertools

# 通过chain方法从可迭代对象中生成；最后展开成列表

sum_data = set(itertools.chain.from_iterable(results))
sum_data
final_df1 = df_fragments[df_fragments.index.isin(sum_data)]

In [51]:
final_df

,chr,start,end,barcode,count
13907500,chr1,4768517,4768564,GTAGTTCGTGTCAAATATCG,1
11413863,chr1,4768517,4768576,GCCAACGATCTTCCGGGGGT,2
18842260,chr1,4768519,4768566,TTGCGAGCATCAAGCCTATC,1
12372524,chr1,4768548,4768778,GCTTAGTATCCGGAATTACA,1
5208374,chr1,4768550,4768613,CAAGGATCGCTATCACGTCC,3
...,...,...,...,...,...
5602607,chr1,21963758,21963790,CAGACTTGGATTGGACCCTT,1
11017400,chr1,21963758,21963803,GATCTTTCCAATCGCAATTT,2
5905439,chr1,21963768,21963808,CAGTACATACGTCGCGGCTT,1
4861369,chr1,21963777,21963814,ATTACAAGACGATGTAACTA,1


In [76]:
final_df1

,chr,start,end,barcode,count
13907500,chr1,4768517,4768564,GTAGTTCGTGTCAAATATCG,1
11413863,chr1,4768517,4768576,GCCAACGATCTTCCGGGGGT,2
18842260,chr1,4768519,4768566,TTGCGAGCATCAAGCCTATC,1
12372524,chr1,4768548,4768778,GCTTAGTATCCGGAATTACA,1
5208374,chr1,4768550,4768613,CAAGGATCGCTATCACGTCC,3
...,...,...,...,...,...
16669,chrY,90808875,90808917,AACAATATGAGCAGGCTGCA,1
12776628,chrY,90808877,90808912,GGCATTGCAAGTGTTGGTCT,1
247127,chrY,90808877,90808917,AACGCGAGACTGACGGATAA,1
7950134,chrY,90808878,90808917,CGATGGGCGGAATTAGGGTC,1


In [ ]:
list(final_df['count']) == list(final_df1['count'])

In [54]:
ad = final_df == final_df1

In [65]:
ad['count'].value_counts()

In [89]:
final_df_count = final_df1.groupby('barcode', as_index=False).agg({'count': 'sum'})

In [92]:
final_df_count

,barcode,count
0,AACAATAACGTCTCAACAAT,2
1,AACAATAACGTCTCACACAC,1
2,AACAATAACGTCTCAGCCCA,2
3,AACAATAACGTCTCAGGAAA,1
4,AACAATAACGTCTCAGGGTC,8
...,...,...
566282,TTTTTGTTGTCCGATGCGGG,6
566283,TTTTTGTTGTCCGATTCCCT,1
566284,TTTTTGTTGTCCGATTGACC,38
566285,TTTTTGTTGTCCGATTGCGA,2


In [93]:
df_barcode = pd.merge(df_barcode, final_df_count, on='barcode', how='outer').fillna(0)

In [94]:
df_barcode

,barcode,fragments,fragments_overlapping_promoter,count
0,AACAATAACGTCTCAACAAT,3,0,2.0
1,AACAATAACGTCTCAACGCG,1,1,0.0
2,AACAATAACGTCTCACACAC,1,1,1.0
3,AACAATAACGTCTCACAGTG,1,0,0.0
4,AACAATAACGTCTCACGTCC,1,0,0.0
...,...,...,...,...
668319,TTTTTGTTGTCCGATGCGGG,6,6,6.0
668320,TTTTTGTTGTCCGATTCCCT,3,2,1.0
668321,TTTTTGTTGTCCGATTGACC,67,31,38.0
668322,TTTTTGTTGTCCGATTGCGA,9,4,2.0


In [95]:
df_barcode = df_barcode.rename(columns={'count': 'overlap_peaks'})

In [96]:
df_barcode['frac_peak'] = round(df_barcode['overlap_peaks'] / df_barcode['fragments'], 4)

In [97]:
df_barcode

,barcode,fragments,fragments_overlapping_promoter,overlap_peaks,frac_peak
0,AACAATAACGTCTCAACAAT,3,0,2.0,0.6667
1,AACAATAACGTCTCAACGCG,1,1,0.0,0.0000
2,AACAATAACGTCTCACACAC,1,1,1.0,1.0000
3,AACAATAACGTCTCACAGTG,1,0,0.0,0.0000
4,AACAATAACGTCTCACGTCC,1,0,0.0,0.0000
...,...,...,...,...,...
668319,TTTTTGTTGTCCGATGCGGG,6,6,6.0,1.0000
668320,TTTTTGTTGTCCGATTCCCT,3,2,1.0,0.3333
668321,TTTTTGTTGTCCGATTGACC,67,31,38.0,0.5672
668322,TTTTTGTTGTCCGATTGCGA,9,4,2.0,0.2222


In [ ]:
df_barcode['cell_called'] = df_barcode['barcode'].apply(lambda x: True if x in cell_barcode else False)

In [98]:
df_fragments

,chr,start,end,barcode,count
165035,chr1,3003786,3003903,AACAATTGGCTTCTTTTGAC,1
1633690,chr1,3003975,3004005,ACCCTTCAGGATCATGACCC,1
10348822,chr1,3004652,3004699,GAAAAGTGTGCGTACCCGCC,1
2386893,chr1,3004791,3004834,ACTGGTTGCATCCTAGCCCA,2
15793733,chr1,3005277,3005613,TCAACTTAGTGAGCACGTCC,3
...,...,...,...,...,...
9833889,chrY,90828968,90829015,CTTCAGGCGATGTTTCATTC,1
3749131,chrY,90828971,90829014,AGTCTGTTCTAGGCCAGTAC,3
502508,chrY,90828984,90829202,AAGTGTACGTTGCAAGGGTC,2
15917205,chrY,90829080,90829202,TCATTCATGGTGGAACACAC,1


In [99]:
frag_chr = df_fragments[df_fragments["chr"] == "chr1"]

In [101]:
frag_overlap_peak = frag_chr[ (frag_chr["start"] >= 4768517) & (frag_chr["end"] <= 4768564)]

In [102]:
frag_overlap_peak

,chr,start,end,barcode,count
13907500,chr1,4768517,4768564,GTAGTTCGTGTCAAATATCG,1


In [104]:
set(df_peaks.chr)

{'chr1',
 'chr10',
 'chr11',
 'chr12',
 'chr13',
 'chr14',
 'chr15',
 'chr16',
 'chr17',
 'chr18',
 'chr19',
 'chr2',
 'chr3',
 'chr4',
 'chr5',
 'chr6',
 'chr7',
 'chr8',
 'chr9',
 'chrX',
 'chrY'}

In [105]:
df_peaks1=pd.read_csv("/SGRNJ06/randd/PROJECT/scATAC/20231006_ME_sc/A0920_3_ME_YSPB_WX_D_T7_EDTA5030/03.atac/A0920_3_ME_YSPB_WX_D_T7_EDTA5030/outs/filtered_peak_bc_matrix/peaks.bed",header=None, sep='\t', names=["chr", "start", "end"])

In [106]:
df_peaks1

,chr,start,end
0,chr1,3094558,3095456
1,chr1,3119609,3120531
2,chr1,3121272,3121966
3,chr1,3297177,3298080
4,chr1,3395281,3396216
...,...,...,...
181725,GL456216.1,40484,41157
181726,GL456216.1,43802,44679
181727,GL456216.1,48811,49704
181728,JH584292.1,12577,13540


In [107]:
set(df_peaks1.chr)

{'GL456216.1',
 'GL456233.1',
 'JH584292.1',
 'JH584295.1',
 'JH584304.1',
 'chr1',
 'chr10',
 'chr11',
 'chr12',
 'chr13',
 'chr14',
 'chr15',
 'chr16',
 'chr17',
 'chr18',
 'chr19',
 'chr2',
 'chr3',
 'chr4',
 'chr5',
 'chr6',
 'chr7',
 'chr8',
 'chr9',
 'chrX',
 'chrY'}